## DeepFace

[**DeepFace**](https://github.com/serengil/deepface) is a lightweight face recognition and facial attribute analysis package for Python. You can apply facial analysis with just a few lines of code. It is fully open source and available on PyPI. All you need is to call **pip install deepface** command.

It supports the most popular face recognition models including **VGG-Face**, **Google FaceNet**, **OpenFace**, **Facebook DeepFace**,**DeepID**, **Dlib** and **ArcFace**. Besides, it can analyze facial attributes such as **emotion**, **age**, **gender** and **race** prediction as well in its facial attribute analysis module.

GitHub repo: https://github.com/serengil/deepface

There are many ways to support a project - starring⭐️ it is just one.

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import zipfile
import cv2
import tqdm
import matplotlib.pyplot as plt
import itertools

In [ ]:
ls


Deepface will download pre-trained weights from Google Drive source but I copied weights in the inputs to skip downloading step

In [ ]:
home = str(Path.home())
print("HOME_FOLDER is ", home)

if not os.path.exists(home+"/.deepface"):
    os.mkdir(home+"/.deepface")
    
if not os.path.exists(home+"/.deepface/weights"):    
    os.mkdir(home+"/.deepface/weights")

#os.listdir('../input/pretrained-models')

import shutil

shutil.copy("../input/pretrained-models/vgg_face_weights.h5", home+"/.deepface/weights")
shutil.copy("../input/pretrained-models/facenet_weights.h5", home+"/.deepface/weights")
shutil.copy("../input/pretrained-models/arcface_weights.h5", home+"/.deepface/weights")

In [ ]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [ ]:
TRAIN_ZIP = '../input/recognizing-faces-in-the-wild/train.zip'
TEST_ZIP='../input/recognizing-faces-in-the-wild/test.zip'

In [ ]:
print("unzipping train set")
with zipfile.ZipFile(TRAIN_ZIP, 'r') as zip_ref:
    zip_ref.extractall("../output/kaggle/working/train")

print("unzipping test set")
with zipfile.ZipFile(TEST_ZIP, 'r') as zip_ref:
    zip_ref.extractall("../output/kaggle/working/test")

In [ ]:
df = pd.read_csv("../input/recognizing-faces-in-the-wild/train_relationships.csv")
df['related'] = 1
df.head()

Train relationships file show folders for related ones. A folder might contain multiple photos.

In [ ]:
print(len(df))

In [ ]:
((df['p1'] == 'F0002/MID3') & (df['p2'] == 'F0002/MID1')).any()

In [ ]:
def findCustomImages(path):
    images = []
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            images.append(os.path.join(dirname, filename))
    
    return images

def getAllFolders(path):
    _, F, _ = next(os.walk(path))
    F.sort()
    return F

In [ ]:
def addUnrelatedRelations(df, root):
    
    #create empty data-frame for all non-relations
    non_relations = []
    
    #get all families
    F = getAllFolders(root)
    for f in F:
        family_members = getAllFolders(root+f)
        
        #correct format for family members
        for i in range(len(family_members)):
            family_members[i] = f + '/' + family_members[i]
            
        #check if family-relation already exists, if not add to non_relations list
        for subset in itertools.combinations(family_members, 2):
            if not ((df['p1'] == subset[0]) & (df['p2'] == subset[1])).any() and not ((df['p1'] == subset[1]) & (df['p2'] == subset[0])).any():
                non_relations.append([subset[0], subset[1], 0])
    
    return pd.DataFrame(non_relations, columns = ['p1', 'p2', 'related'])


In [ ]:
non_relations_df = addUnrelatedRelations(df, "../output/kaggle/working/train/")

In [ ]:
new_df = pd.concat([df, non_relations_df]).sort_values(by=['p1']).reset_index(drop=True)

In [ ]:
root = "../output/kaggle/working/train/"

samples = []
for index, instance in new_df.iterrows():
    person1 = root+instance.p1
    person2 = root+instance.p2

    person1_images = findCustomImages(person1)
    person2_images = findCustomImages(person2)
    
    for i in person1_images:
        for j in person2_images:
            sample = []
            sample.append(i)
            sample.append(j)
            sample.append(instance['related'])
            samples.append(sample)

In [ ]:
df = pd.DataFrame(samples, columns = ["p1", "p2", "related"])

Now, data frame has unique photos of related ones.

In [ ]:
df.head(15)

In [ ]:
print("There are ",df.shape[0]," image pairs existing as a relative")

# DeepFace Framework for Python

In [ ]:
!pip install gdown --no-deps
!pip install mtcnn --no-deps
!pip install deepface --no-deps

In [ ]:
from deepface import DeepFace

# Face Recognition

In [ ]:
model_names = [ "Facenet512", "VGG-Face", "Facenet", "OpenFace", "DeepFace", "ArcFace", "Dlib"]
metrics = ["cosine", "euclidean_l2"]

In [ ]:
# df = df.sample(100)

In [ ]:
for model_name in model_names:
    
    model = DeepFace.build_model(model_name)
    
    for metric in metrics:
    
        obj = DeepFace.verify(df[['p1', 'p2']].values.tolist(), model_name = model_name, model = model
                                           , distance_metric = metric, enforce_detection = False)
    
        distances = []
        for key in obj.keys():
            distance = obj[key]["distance"]
            distances.append(distance)

        df["%s_%s" % (model_name, metric)] = distances

In [ ]:
df.head()

In [ ]:
for model in model_names:
    for metric in metrics:
        print("Distribution for ",model," and ", metric," pair")
        df['%s_%s' % (model, metric)].plot.kde()
        plt.show()
        print("-----------------------------------------")